# Advanced Chunking Strategies

In [ ]:
! pip install -U chromadb langchain llama-index langchain_experimental langchain_openai


In [1]:
from rich import print
from langchain.docstore.document import Document
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [2]:
local_llm = ChatOllama(model="mistral")

/var/folders/5g/452dgvq955l5p94xk82bt6x80000gn/T/ipykernel_39480/136981833.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  local_llm = ChatOllama(model="mistral")


In [3]:
# RAG
def rag(chunks, collection_name):
    vectorstore = Chroma.from_documents(
        documents=documents,
        collection_name=collection_name,
        embedding=embeddings.ollama.OllamaEmbeddings(model='nomic-embed-text'),
    )
    retriever = vectorstore.as_retriever()

    prompt_template = """Answer the question based only on the following context:
    {context}
    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(prompt_template)

    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | local_llm
        | StrOutputParser()
    )
    result = chain.invoke("What is the use of Text Splitting?")
    print(result)

In [4]:
# 1. Character Text Splitting
print("#### Character Text Splitting ####")

text = "Text splitting in LangChain is a critical feature that facilitates the division of large texts into smaller, manageable segments. "

# Manual Splitting
chunks = []
chunk_size = 35 # Characters
for i in range(0, len(text), chunk_size):
    chunk = text[i:i + chunk_size]
    chunks.append(chunk)
documents = [Document(page_content=chunk, metadata={"source": "local"}) for chunk in chunks]
print(documents)

#### Character Text Splitting ####

[
    Document(metadata={'source': 'local'}, page_content='Text splitting in LangChain is a cr'),
    Document(metadata={'source': 'local'}, page_content='itical feature that facilitates the'),
    Document(metadata={'source': 'local'}, page_content=' division of large texts into small'),
    Document(metadata={'source': 'local'}, page_content='er, manageable segments. ')
]

In [5]:
# Automatic Text Splitting
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size = 35, chunk_overlap=0, separator='', strip_whitespace=False)
documents = text_splitter.create_documents([text])
print(documents)


[
    Document(metadata={}, page_content='Text splitting in LangChain is a cr'),
    Document(metadata={}, page_content='itical feature that facilitates the'),
    Document(metadata={}, page_content=' division of large texts into small'),
    Document(metadata={}, page_content='er, manageable segments. ')
]

"Recursive Character Text Splitting" is a technique used in Natural Language Processing (NLP) to break down large text documents into smaller, manageable chunks by repeatedly splitting the text based on specific characters, like newlines or spaces, while prioritizing keeping semantically related content together, essentially preserving context within the chunks;

In [7]:
# 2. Recursive Character Text Splitting
print("#### Recursive Character Text Splitting ####")

from langchain.text_splitter import RecursiveCharacterTextSplitter
with open('content.txt', 'r', encoding='utf-8') as file:
    text = file.read()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 65, chunk_overlap=0) # ["\n\n", "\n", " ", ""] 65,450
print(text_splitter.create_documents([text])) 


#### Recursive Character Text Splitting ####

[
    Document(metadata={}, page_content='"Recursive Character Text Splitting" is a technique used in'),
    Document(metadata={}, page_content='Natural Language Processing (NLP) to break down large text'),
    Document(metadata={}, page_content='documents into smaller, manageable chunks by repeatedly'),
    Document(metadata={}, page_content='splitting the text based on specific characters, like newlines'),
    Document(metadata={}, page_content='or spaces, while prioritizing keeping semantically related'),
    Document(metadata={}, page_content='content together, essentially preserving context within the'),
    Document(metadata={}, page_content='chunks; this is achieved by recursively splitting the text until'),
    Document(metadata={}, page_content='the desired chunk size is reached, starting with larger'),
    Document(metadata={}, page_content='separators like double newlines and then progressively moving to'),
    Document(metadata={}, page_content='smaller ones like single newlines and spaces.'),
    Document(metadata={}, page_content='Key points about Recursive Character Text Splitting:'),
    Document(metadata={}, page_content='Context Preservation:'),
    Document(metadata={}, page_content='The primary goal is to maintain the meaning of the text by'),
    Document(metadata={}, page_content='splitting at points where the content naturally breaks, like'),
    Document(metadata={}, page_content='between paragraphs or sentences, rather than arbitrarily'),
    Document(metadata={}, page_content='chopping the text.'),
    Document(metadata={}, page_content='Hierarchical Splitting:'),
    Document(metadata={}, page_content='It uses a recursive approach, meaning it repeatedly splits the'),
    Document(metadata={}, page_content='text based on a hierarchy of separators, typically starting with'),
    Document(metadata={}, page_content='larger separators (like double newlines) and then moving to'),
    Document(metadata={}, page_content='smaller ones (like single newlines and spaces).'),
    Document(metadata={}, page_content='Customizable Delimiters:'),
    Document(metadata={}, page_content='You can specify which characters to use as splitting points'),
    Document(metadata={}, page_content='based on your specific needs.'),
    Document(metadata={}, page_content='Application in NLP:'),
    Document(metadata={}, page_content='This technique is commonly used in situations where you need to'),
    Document(metadata={}, page_content='process large text documents in smaller chunks, such as when'),
    Document(metadata={}, page_content='feeding text into a language model for analysis or question'),
    Document(metadata={}, page_content='answering')
]

In [8]:
# 3. Document Specific Splitting
print("#### Document Specific Splitting ####")

# Document Specific Splitting - Markdown
from langchain.text_splitter import MarkdownTextSplitter
splitter = MarkdownTextSplitter(chunk_size = 40, chunk_overlap=0)
markdown_text = """
# Fun in California

## Driving

Try driving on the 1 down to San Diego

### Food

Make sure to eat a burrito while you're there

## Hiking

Go to Yosemite
"""
print(splitter.create_documents([markdown_text]))

#### Document Specific Splitting ####

[
    Document(metadata={}, page_content='# Fun in California\n\n## Driving'),
    Document(metadata={}, page_content='Try driving on the 1 down to San Diego'),
    Document(metadata={}, page_content='### Food'),
    Document(metadata={}, page_content="Make sure to eat a burrito while you're"),
    Document(metadata={}, page_content='there'),
    Document(metadata={}, page_content='## Hiking\n\nGo to Yosemite')
]

In [9]:
# Document Specific Splitting - Python
from langchain.text_splitter import PythonCodeTextSplitter
python_text = """
class Person:
  def __init__(self, name, age):
    self.name = name
    self.age = age

p1 = Person("John", 36)

for i in range(10):
    print (i)
"""
python_splitter = PythonCodeTextSplitter(chunk_size=100, chunk_overlap=0)
print(python_splitter.create_documents([python_text]))


[
    Document(
        metadata={},
        page_content='class Person:\n  def __init__(self, name, age):\n    self.name = name\n    self.age = age'
    ),
    Document(metadata={}, page_content='p1 = Person("John", 36)\n\nfor i in range(10):\n    print (i)')
]

In [10]:
# Document Specific Splitting - Javascript
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
javascript_text = """
// Function is called, the return value will end up in x
let x = myFunction(4, 3);

function myFunction(a, b) {
// Function returns the product of a and b
  return a * b;
}
"""
js_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.JS, chunk_size=65, chunk_overlap=0
)
print(js_splitter.create_documents([javascript_text]))

[
    Document(metadata={}, page_content='// Function is called, the return value will end up in x'),
    Document(metadata={}, page_content='let x = myFunction(4, 3);'),
    Document(metadata={}, page_content='function myFunction(a, b) {'),
    Document(metadata={}, page_content='// Function returns the product of a and b\n  return a * b;\n}')
]

# 4. Semantic Chunking

Sematic Chunking: Semantic Chunking considers the relationships within the text. It divides the text into meaningful, semantically complete chunks. This approach ensures the information’s integrity during retrieval, leading to a more accurate and contextually appropriate outcome. It is slower compared to the previous chunking strategy

In [ ]:
print("#### Semantic Chunking ####")

from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# Percentile - all differences between sentences are calculated, and then any difference greater than the X percentile is split
text_splitter = SemanticChunker(OpenAIEmbeddings())
text_splitter = SemanticChunker(
    OpenAIEmbeddings(), breakpoint_threshold_type="percentile" # "standard_deviation", "interquartile"
)
documents = text_splitter.create_documents([text])
print(documents)

# 5. Agentic Chunking

Agentic Chunk: The hypothesis here is to process documents in a fashion that humans would do.

We start at the top of the document, treating the first part as a chunk.
We continue down the document, deciding if a new sentence or piece of information belongs with the first chunk or should start a new one
We keep this up until we reach the end of the document.
This approach is still being tested and isn't quite ready for the big leagues due to the time it takes to process multiple LLM calls and the cost of those calls. There's no implementation available in public libraries just yet.

In [13]:
print("#### Proposition-Based Chunking ####")

# https://arxiv.org/pdf/2312.06648.pdf

from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.chains import create_extraction_chain
from typing import Optional, List
from langchain.chains import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel
from langchain import hub

obj = hub.pull("wfh/proposal-indexing")
llm = ChatOpenAI(model='gpt-3.5-turbo')
runnable = obj | llm


#### Proposition-Based Chunking ####

/Users/rajeshthakur/miniconda3/envs/ai_experiments/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
class Sentences(BaseModel):
    sentences: List[str]


# Extraction
extraction_chain = create_extraction_chain_pydantic(pydantic_schema=Sentences, llm=llm)
def get_propositions(text):
    runnable_output = runnable.invoke({
    	"input": text
    }).content
    propositions = extraction_chain.invoke(runnable_output)["text"][0].sentences
    return propositions

paragraphs = text.split("\n\n")
text_propositions = []
for i, para in enumerate(paragraphs[:5]):
    propositions = get_propositions(para)
    text_propositions.extend(propositions)
    print (f"Done with {i}")

print (f"You have {len(text_propositions)} propositions")
print(text_propositions[:10])

print("#### Agentic Chunking ####")

from agentic_chunk import AgenticChunker
ac = AgenticChunker()
ac.add_propositions(text_propositions)
print(ac.pretty_print_chunks())
chunks = ac.get_chunks(get_type='list_of_strings')
print(chunks)
documents = [Document(page_content=chunk, metadata={"source": "local"}) for chunk in chunks]
rag(documents, "agentic-chunks")

/var/folders/5g/452dgvq955l5p94xk82bt6x80000gn/T/ipykernel_39480/1524913369.py:6: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` thatis available on ChatModels capable of tool calling.You can read more about the method here: <https://python.langchain.com/docs/modules/model_io/chat/structured_output/>. Please follow our extraction use case documentation for more guidelineson how to do information extraction with LLMs.<https://python.langchain.com/docs/use_cases/extraction/>. If you notice other issues, please provide feedback here:<https://github.com/langchain-ai/langchain/discussions/18154>
  extraction_chain = create_extraction_chain_pydantic(pydantic_schema=Sentences, llm=llm)
/Users/rajeshthakur/miniconda3/envs/ai_experiments/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:777: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `Sentences` to V2.
  warn(


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************xC0A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}